In [3]:
import pandas as pd
import recordlinkage
from graphdatascience import GraphDataScience
from getpass import getpass
from time import perf_counter

# Read emails

In [24]:
email_df = pd.read_csv("../data/emails", header=None)
email_df.columns=['email']

# Set up blocking columns on first two letters, top level domain, and remaining letters.

In [25]:
email_df['first_letter'] = email_df['email'].str[:1]
email_df['first_letters'] = email_df['email'].str[:2]

In [26]:
email_df['tld'] = email_df['email'].str.split(".").map(lambda x: x[-1])

In [27]:
email_df['middle_letters'] = email_df['email'].str[2:].str.split(".").map(lambda x: ".".join(x[:-1]))

In [28]:
email_df.head()

,email,first_letter,first_letters,tld,middle_letters
0,cooperjames@yahoo.com,c,co,com,operjames@yahoo
1,cindy24@hotmail.com,c,ci,com,ndy24@hotmail
2,averyjohn@ruiz.com,a,av,com,eryjohn@ruiz
3,santanachristopher@burns-robinson.com,s,sa,com,ntanachristopher@burns-robinson
4,alvarezsherri@gmail.com,a,al,com,varezsherri@gmail


# Identify candidate links

Blocking on first letter only causes my kernel to crash. **Expect the next cell to fail unless you have a lot of memory!**

In [ ]:
indexer = recordlinkage.Index()
indexer.block("first_letter")
candidate_links = indexer.index(email_df)

Try blocking on the first two letters + the tld or the middle letters.

In [8]:
start_time = perf_counter()
indexer = recordlinkage.Index()
indexer.block(["first_letters", "tld"])
indexer.block("middle_letters")
candidate_links = indexer.index(email_df)
end_time = perf_counter()
print(end_time - start_time)

31.721396744999765


In [9]:
len(candidate_links)

64816224

In [10]:
start_time = perf_counter()
compare = recordlinkage.Compare()
compare.string(
    "email", "email", method="levenshtein", label="email"
)
features = compare.compute(candidate_links, email_df)
end_time = perf_counter()
print(end_time - start_time)

577.5809937390004


In [11]:
matched_df = features[features['email']>0.8].reset_index().copy()

In [12]:
matched_df.columns = ['index1', 'index2', 'score']

In [13]:
matched_df = matched_df.merge(email_df['email'], left_on='index1', right_index=True)
matched_df = matched_df.merge(email_df['email'], left_on='index2', right_index=True)

In [14]:
matched_df.shape

(455781, 5)

In [5]:
neo4j_password = getpass("neo4j password")

neo4j password ········


In [6]:
gds = GraphDataScience("neo4j+s://3bddbcd7.databases.neo4j.io", auth=("neo4j", neo4j_password))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


In [22]:
gds.run_cypher("""
CALL {
UNWIND $data AS row
MATCH (e1:Email {address:row['email_x']})
MATCH (e2:Email {address:row['email_y']})
WITH e1, e2, row, apoc.text.levenshteinDistance(e1.address, e2.address) AS levenshteinDistance
WHERE levenshteinDistance <= 2
MERGE (e1)-[r:HAS_SIMILAR_ADDRESS]->(e2)
SET r.recordLinkageScore = row['score'],
r.levenshteinDistance = levenshteinDistance
} IN TRANSACTIONS OF 10000 ROWS""",
              {"data": matched_df[['score', 'email_x', 'email_y']].to_dict("records")})

Failed to read from defunct connection IPv4Address(('3bddbcd7.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.28.32.244', 7687)))


""


# Compare record linkage results with embedding results

In [7]:
gds.run_cypher(
"""MATCH (e1)-[r]-(e2)
WHERE e1.address < e2.address
WITH e1, e2, 
max(CASE WHEN r:HAS_SIMILAR_ADDRESS THEN 1 ELSE 0 END) AS recordLinkageMatch, 
max(CASE WHEN r:HAS_SIMILAR_EMBEDDING THEN 1 ELSE 0 END) AS embeddingLinkageMatch
WITH e1, e2, 
CASE WHEN recordLinkageMatch = 1 and embeddingLinkageMatch = 1 THEN "both"
WHEN recordLinkageMatch = 1 AND embeddingLinkageMatch = 0 THEN "record linkage only"
WHEN recordLinkageMatch = 0 AND embeddingLinkageMatch = 1 THEN "embedding similarity only" END
AS matchType
return matchType, count(*) AS pairs""")

,matchType,pairs
0,both,248790
1,embedding similarity only,15237
2,record linkage only,4106


Look at examples that matched on embedding similarity only.

In [8]:
gds.run_cypher(
"""MATCH (e1)-[r]-(e2)
WHERE e1.address < e2.address
WITH e1, e2, 
max(CASE WHEN r:HAS_SIMILAR_ADDRESS THEN 1 ELSE 0 END) AS recordLinkageMatch, 
max(CASE WHEN r:HAS_SIMILAR_EMBEDDING THEN 1 ELSE 0 END) AS embeddingLinkageMatch,
collect(r) AS rels
WITH e1, e2, rels,
CASE WHEN recordLinkageMatch = 1 and embeddingLinkageMatch = 1 THEN "both"
WHEN recordLinkageMatch = 1 AND embeddingLinkageMatch = 0 THEN "record linkage only"
WHEN recordLinkageMatch = 0 AND embeddingLinkageMatch = 1 THEN "embedding similarity only" END
AS matchType
WHERE matchType = "embedding similarity only"
return e1.address, e2.address
limit 4""")

,e1.address,e2.address
0,amills@yahoo.com,zmiles@yahoo.com
1,amills@yahoo.com,awells@yahoo.com
2,ahill@yahoo.com,amills@yahoo.com
3,jward@gmail.com,lware@gmail.com


Create vector index.

In [9]:
gds.run_cypher("""
CREATE VECTOR INDEX emailEditEmbedding IF NOT EXISTS
FOR (e:Email)
ON e.editEmbedding
OPTIONS {indexConfig: {
 `vector.dimensions`: 128,
 `vector.similarity_function`: 'cosine'
}}""")

""


Find Emails that record linkage found but embedding similarity missed. Check their 80 nearest neighbors and compare with the missed match.

In [14]:
gds.run_cypher("""
MATCH (e)-[r]-(e2)
WITH e, e2, 
max(CASE WHEN r:HAS_SIMILAR_ADDRESS THEN 1 ELSE 0 END) AS recordLinkageMatch, 
max(CASE WHEN r:HAS_SIMILAR_EMBEDDING THEN 1 ELSE 0 END) AS embeddingSimilarityMatch
WITH e, e2, gds.similarity.cosine(e.editEmbedding, e2.editEmbedding) AS cosineSimilarity
WHERE recordLinkageMatch = 1 AND embeddingSimilarityMatch = 0
CALL {
    WITH e, e2, cosineSimilarity
    RETURN e2.address AS address2, cosineSimilarity AS similarity 
    ORDER BY similarity DESC
    limit 1
}
CALL {
  WITH e
  CALL db.index.vector.queryNodes("emailEditEmbedding", 81, e.editEmbedding) YIELD node, score
  WITH
  apoc.text.levenshteinDistance(e.address, node.address) AS levenshteinDistance,
  node, score
  ORDER BY score 
  WITH sum(CASE WHEN levenshteinDistance <= 2 THEN 1 else 0 END) AS neighborsEditLessThan2,
  min(score) AS similarityToNeighbor80,
  collect(levenshteinDistance) AS editDistances
  RETURN similarityToNeighbor80, editDistances[0] AS editDistanceToNeighbor80, neighborsEditLessThan2
}
RETURN e.address AS address, address2, similarity AS similarityToMissedMatch, 
similarityToNeighbor80, editDistanceToNeighbor80, neighborsEditLessThan2
ORDER BY similarityToNeighbor80 - similarityToMissedMatch LIMIT 10""")

,address,address2,similarityToMissedMatch,similarityToNeighbor80,editDistanceToNeighbor80,neighborsEditLessThan2
0,dbowen@hotmail.com,dbowers@hotmail.com,0.912871,0.958400,5,10
1,james87@hotmail.com,james41@hotmail.com,0.909403,0.954953,4,54
2,james60@hotmail.com,james94@hotmail.com,0.909852,0.955539,2,53
3,michael24@hotmail.com,michael30@hotmail.com,0.908219,0.954174,2,71
4,james17@hotmail.com,james94@hotmail.com,0.909470,0.955590,2,56
5,michael30@hotmail.com,michael24@hotmail.com,0.908219,0.954387,2,58
6,james45@hotmail.com,james37@hotmail.com,0.908316,0.954562,5,55
7,dbowers@hotmail.com,dbowen@hotmail.com,0.912871,0.959227,6,6
8,james45@hotmail.com,james60@hotmail.com,0.907734,0.954562,5,55
9,michael68@hotmail.com,michael40@hotmail.com,0.906522,0.953381,5,64
